In [ ]:
from blend_models import blend
import os
import re
from typing import List, Optional, Tuple, Union
from gen_images import make_transform
import dnnlib
import numpy as np
import PIL.Image
import torch

import legacy

In [ ]:
outdir = 'samples'

device = torch.device('cuda')

G = blend('models/stylegan3-r-ffhq-1024x1024.pkl', 'models/stylegan3-r-ffhq-1024x1024-cartoon.pkl', 5)
os.makedirs(outdir, exist_ok=True)

# Generate images.
seeds = [1, 2, 3, 4, 5, 6]
for seed_idx, seed in enumerate(seeds):
    print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
    z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)

    # Construct an inverse rotation/translation matrix and pass to the generator.  The
    # generator expects this matrix as an inverse to avoid potentially failing numerical
    # operations in the network.
    if hasattr(G.synthesis, 'input'):
        m = make_transform((0, 0), 0)
        m = np.linalg.inv(m)
        G.synthesis.input.transform.copy_(torch.from_numpy(m))

    img = G(z, torch.zeros([1, G.c_dim], device=device), truncation_psi=1, noise_mode='const')
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seed:04d}.png')
